In [1]:
import pandas as pd

In [42]:
# 메인 데이터
app = pd.read_csv("./score_data/application_train_EXT_SOURCE.csv")

# 서브 테이블들
bureau = pd.read_csv("./score_data/bureau_final.csv")
credit = pd.read_csv("./score_data/credit_card_kpi_final.csv")
installment = pd.read_csv("./score_data/installment_kpi_final.csv")
pos = pd.read_csv("./score_data/pos_final.csv")
prev = pd.read_csv("./score_data/previous_final.csv")
bureau_active_agg = pd.read_csv("./score_data/df2_active_agg.csv")

In [ ]:
credit['AVG_SK_DPD']

In [ ]:
pos['AVG_SK_DPD']

In [65]:
credit = credit.rename(columns={"AVG_SK_DPD": "CREDIT_AVG_SK_DPD"})
pos = pos.rename(columns={"AVG_SK_DPD": "POS_AVG_SK_DPD"})

In [43]:
app.shape

(307511, 5)

In [ ]:
# # 순차 병합
# df = app.merge(credit, on="SK_ID_CURR", how="left") \
#         .merge(installment, on="SK_ID_CURR", how="left") \
#         .merge(pos_agg, on="SK_ID_CURR", how="left") \
#         .merge(prev, on="SK_ID_CURR", how="left")

# print("병합 완료. shape:", df.shape)

병합 완료. shape: (307511, 28)


In [32]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 13 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   SK_ID_CURR                     int64  
 1   SK_ID_BUREAU                   int64  
 2   IS_OVERDUE_LONG                int64  
 3   IS_OVERDUE_SHORT               int64  
 4   END_STATUS                     object 
 5   HAS_HIGH_RISK_DEBT             object 
 6   IS_LONG_TERM_ACTIVE            int64  
 7   DELTA_CREDIT_SUM_IF_INCREASED  float64
 8   DELTA_CREDIT_PCT_IF_INCREASED  float64
 9   CREDIT_INCREASED               bool   
 10  OVERLAP                        bool   
 11  OVERLAP_YEARS                  float64
 12  HAS_BAD_STATUS_36M             float64
dtypes: bool(2), float64(4), int64(5), object(2)
memory usage: 147.3+ MB


In [36]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,IS_OVERDUE_LONG,IS_OVERDUE_SHORT,END_STATUS,HAS_HIGH_RISK_DEBT,IS_LONG_TERM_ACTIVE,DELTA_CREDIT_SUM_IF_INCREASED,DELTA_CREDIT_PCT_IF_INCREASED,CREDIT_INCREASED,OVERLAP,OVERLAP_YEARS,HAS_BAD_STATUS_36M
0,215354,5714462,0,0,정시종료,Non-High Risk,0,NaN,NaN,False,True,4.641096,NaN
1,215354,5714463,0,0,NaN,Non-High Risk,1,45000.0,0.25000,True,True,75.802740,NaN
2,215354,5714464,0,0,NaN,Non-High Risk,1,239323.5,1.06366,True,True,3.501370,NaN
3,215354,5714465,0,0,NaN,Non-High Risk,0,NaN,NaN,False,True,2.002740,NaN
4,215354,5714466,0,0,NaN,Non-High Risk,1,NaN,NaN,False,False,-0.550685,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,0,0,NaN,Non-High Risk,0,NaN,NaN,False,True,0.800000,NaN
1716424,100044,5057754,0,0,조기상환,Non-High Risk,0,NaN,NaN,False,True,2.805479,NaN
1716425,100044,5057762,0,0,연체종료,Non-High Risk,0,NaN,NaN,False,True,6.764384,NaN
1716426,246829,5057770,0,0,정시종료,Non-High Risk,0,NaN,NaN,False,False,-0.506849,NaN


In [38]:
print(bureau['END_STATUS'].value_counts())

END_STATUS
조기상환    485006
정시종료    323616
연체종료    238096
Name: count, dtype: int64


In [ ]:
# 범주형 처리 먼저
bureau["HAS_HIGH_RISK_DEBT"] = bureau["HAS_HIGH_RISK_DEBT"].map({"High Risk": 1, "Non-High Risk": 0})
bureau["CREDIT_INCREASED"] = bureau["CREDIT_INCREASED"].astype(int)
bureau["OVERLAP"] = bureau["OVERLAP"].astype(int)

# END_STATUS 위험 점수 예시: 연체종료 1점, 조기상환 -1점, 나머지 0
def map_end_status(x):
    if x == "연체종료":
        return 1
    elif x == "조기상환":
        return -1
    else:
        return 0
bureau["END_STATUS_SCORE"] = bureau["END_STATUS"].map(map_end_status)

# 집계
# agg_bureau = bureau.groupby("SK_ID_CURR").agg({
#     "IS_OVERDUE_LONG": "sum",
#     "IS_OVERDUE_SHORT": "sum",
#     "IS_LONG_TERM_ACTIVE": "sum",
#     "CREDIT_INCREASED": "sum",
#     "DELTA_CREDIT_SUM_IF_INCREASED": "sum",
#     "DELTA_CREDIT_PCT_IF_INCREASED": "mean",
#     "OVERLAP": "sum",
#     "OVERLAP_YEARS": "sum",
#     "HAS_HIGH_RISK_DEBT": "max",
#     "HAS_BAD_STATUS_36M": "max",
#     "END_STATUS_SCORE": "sum"
# }).reset_index()


In [ ]:
# 범주형 처리
bureau["HAS_HIGH_RISK_DEBT"] = bureau["HAS_HIGH_RISK_DEBT"].map({"High Risk": 1, "Non-High Risk": 0})
bureau["CREDIT_INCREASED"] = bureau["CREDIT_INCREASED"].astype(int)
bureau["OVERLAP"] = bureau["OVERLAP"].astype(int)

# END_STATUS 상태별 더미 플래그 생성
bureau["IS_EARLY_REPAID"] = (bureau["END_STATUS"] == "조기상환").astype(int)
bureau["IS_REPAID_ON_TIME"] = (bureau["END_STATUS"] == "정시종료").astype(int)
bureau["IS_OVERDUE_ENDED"] = (bureau["END_STATUS"] == "연체종료").astype(int)
bureau["IS_NOT_ENDED"] = bureau["END_STATUS"].isna().astype(int)

# 집계
agg_bureau = bureau.groupby("SK_ID_CURR").agg({
    "IS_OVERDUE_LONG": "max",              # 장기연체 여부
    "IS_OVERDUE_SHORT": "max",             # 단기연체 여부
    "IS_LONG_TERM_ACTIVE": "max",          
    "CREDIT_INCREASED": "sum",             
    "DELTA_CREDIT_SUM_IF_INCREASED": "sum",
    "DELTA_CREDIT_PCT_IF_INCREASED": "mean",
    "OVERLAP": "sum",
    "OVERLAP_YEARS": "sum",
    "HAS_HIGH_RISK_DEBT": "max",
    "HAS_BAD_STATUS_36M": "max",
    
    # END_STATUS 종료유형별 건수
    "IS_EARLY_REPAID": "sum",
    "IS_REPAID_ON_TIME": "sum",
    "IS_OVERDUE_ENDED": "sum",
    "IS_NOT_ENDED": "sum"
}).reset_index()

# 컬럼 이름
agg_bureau = agg_bureau.rename(columns={
    "IS_EARLY_REPAID": "CNT_EARLY_REPAID",
    "IS_REPAID_ON_TIME": "CNT_REPAID_ON_TIME",
    "IS_OVERDUE_ENDED": "CNT_OVERDUE_ENDED",
    "IS_NOT_ENDED": "CNT_NOT_ENDED"
})

In [83]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 13 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   SK_ID_CURR                     int64  
 1   SK_ID_BUREAU                   int64  
 2   IS_OVERDUE_LONG                int64  
 3   IS_OVERDUE_SHORT               int64  
 4   END_STATUS                     object 
 5   HAS_HIGH_RISK_DEBT             object 
 6   IS_LONG_TERM_ACTIVE            int64  
 7   DELTA_CREDIT_SUM_IF_INCREASED  float64
 8   DELTA_CREDIT_PCT_IF_INCREASED  float64
 9   CREDIT_INCREASED               bool   
 10  OVERLAP                        bool   
 11  OVERLAP_YEARS                  float64
 12  HAS_BAD_STATUS_36M             float64
dtypes: bool(2), float64(4), int64(5), object(2)
memory usage: 147.3+ MB


In [87]:
agg_bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305811 entries, 0 to 305810
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   SK_ID_CURR                     305811 non-null  int64  
 1   IS_OVERDUE_LONG                305811 non-null  int64  
 2   IS_OVERDUE_SHORT               305811 non-null  int64  
 3   IS_LONG_TERM_ACTIVE            305811 non-null  int64  
 4   CREDIT_INCREASED               305811 non-null  int32  
 5   DELTA_CREDIT_SUM_IF_INCREASED  305811 non-null  float64
 6   DELTA_CREDIT_PCT_IF_INCREASED  182814 non-null  float64
 7   OVERLAP                        305811 non-null  int32  
 8   OVERLAP_YEARS                  305811 non-null  float64
 9   HAS_HIGH_RISK_DEBT             305811 non-null  int64  
 10  CNT_EARLY_REPAID               305811 non-null  int32  
 11  CNT_REPAID_ON_TIME             305811 non-null  int32  
 12  CNT_OVERDUE_ENDED             

In [86]:
# HAS_BAD_STATUS_36M 컬럼 삭제
agg_bureau = agg_bureau.drop(columns=["HAS_BAD_STATUS_36M"], errors="ignore")

In [88]:
agg_bureau.to_csv('agg_bureau.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 마스터 파일 불러오기
df_master = pd.read_csv("score_data/merged_kpi_dataset_v2.csv")

# 집계된 bureau 파일 불러오기
# agg_bureau = pd.read_csv("agg_bureau.csv")

# app 기준 ID 추출
valid_ids = set(df_master["SK_ID_CURR"])
agg_bureau_filtered = agg_bureau[agg_bureau["SK_ID_CURR"].isin(valid_ids)].copy()

# 겹치는 컬럼 확인
common_cols = list(set(agg_bureau_filtered.columns) & set(df_master.columns))
common_cols.remove("SK_ID_CURR")  # ID는 남겨야 함

# 덮어쓰기 위해 기존 컬럼 제거
if common_cols:
    df_master.drop(columns=common_cols, inplace=True)

# 병합 (덮어쓰기 처리됨)
df_master = df_master.merge(agg_bureau_filtered, on="SK_ID_CURR", how="left")

# 저장
df_master.to_csv("score_data/merged_kpi_dataset_v3.csv", index=False, encoding="utf-8-sig")
print("병합 완료 → 'score_data/merged_kpi_dataset_v3.csv'")

병합 완료 → 'score_data/merged_kpi_dataset_v3.csv'


In [68]:
# 1. 기존 잘못 붙은 컬럼 제거
df_master = df_master.drop(columns=["AVG_SK_DPD_x", "AVG_SK_DPD_y"], errors="ignore")

# 2. 필요한 컬럼만 추출해서 병합 (rename 없음!)
credit_subset = credit[["SK_ID_CURR", "CREDIT_AVG_SK_DPD"]]
pos_subset = pos[["SK_ID_CURR", "POS_AVG_SK_DPD"]]

# 3. 병합 (ID 기준으로만)
df_master = df_master.merge(credit_subset, on="SK_ID_CURR", how="left")
df_master = df_master.merge(pos_subset, on="SK_ID_CURR", how="left")

# 4. 저장
df_master.to_csv("score_data/merged_kpi_dataset_v3.csv", index=False, encoding="utf-8-sig")
print("AVG_SK_DPD 정리 및 재병합 완료 → v3 저장됨")

AVG_SK_DPD 정리 및 재병합 완료 → v3 저장됨


In [89]:
# 병합: SK_ID_CURR 기준으로 외부 조인
dpd_merged = credit_subset.merge(pos_subset, on="SK_ID_CURR", how="outer")

# 결과 확인
print(dpd_merged.shape)
print(dpd_merged.head())

(340352, 3)
   SK_ID_CURR  CREDIT_AVG_SK_DPD  POS_AVG_SK_DPD
0      100001                NaN        0.777778
1      100002                NaN        0.000000
2      100003                NaN        0.000000
3      100004                NaN        0.000000
4      100005                NaN        0.000000


In [90]:
dpd_merged.to_csv('SK_AVG_DPD.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 마스터 파일 불러오기
df_master = pd.read_csv("score_data/merged_kpi_dataset_v3.csv")

# 기존 잘못 병합된 컬럼 삭제
df_master = df_master.drop(columns=["AVG_SK_DPD_x", "AVG_SK_DPD_y"], errors="ignore")

# 새 DPD 데이터
dpd_merged = pd.read_csv("SK_AVG_DPD.csv")

# 병합 (SK_ID_CURR 기준)
df_master = df_master.merge(dpd_merged, on="SK_ID_CURR", how="left")

In [71]:
# HAS_BAD_STATUS_36M 컬럼 삭제
df_v3 = df_v3.drop(columns=["HAS_BAD_STATUS_36M"], errors="ignore")

# 저장
df_v3.to_csv("score_data/merged_kpi_dataset_v3.csv", index=False, encoding="utf-8-sig")

In [81]:
df_v3 = pd.read_csv('score_data/merged_kpi_dataset_v3.csv')
df_v2 = pd.read_csv('score_data/merged_kpi_dataset_v2.csv')

In [ ]:
df_v3

In [73]:
print(df_v3.info())
print(df_v2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   SK_ID_CURR                     307511 non-null  int64  
 1   EXT_SOURCE_1                   307511 non-null  float64
 2   EXT_SOURCE_2                   307511 non-null  float64
 3   EXT_SOURCE_3                   307511 non-null  float64
 4   TARGET                         307511 non-null  int64  
 5   CC_90PLUS                      86905 non-null   float64
 6   AVG_CARD_USAGE_MONTHS          86905 non-null   float64
 7   AVG_UTIL_RATIO                 86905 non-null   float64
 8   AVG_MIN_PAY_RATIO              86905 non-null   float64
 9   AVG_REPAY_RATIO                86905 non-null   float64
 10  CREDIT_CARD_HISTORY_YEARS      307511 non-null  float64
 11  INST_90PLUS                    291643 non-null  float64
 12  avg_payment_delay             

In [74]:
# v2, v3 컬럼 비교
v2_cols = set(df_v2.columns)
v3_cols = set(df_v3.columns)

# v3에만 있고, v2에는 없는 컬럼
only_in_v3 = v3_cols - v2_cols
print("v3에만 존재하는 컬럼:")
print(sorted(only_in_v3))

v3에만 존재하는 컬럼:
['CNT_EARLY_REPAID', 'CNT_NOT_ENDED', 'CNT_OVERDUE_ENDED', 'CNT_REPAID_ON_TIME', 'CREDIT_AVG_SK_DPD', 'POS_AVG_SK_DPD']


In [75]:
print("공통 컬럼:", sorted(set(df_v2.columns) & set(df_v3.columns)))

공통 컬럼: ['AVG_CARD_USAGE_MONTHS', 'AVG_MIN_PAY_RATIO', 'AVG_REPAY_RATIO', 'AVG_UTIL_RATIO', 'CC_90PLUS', 'CREDIT_CARD_HISTORY_YEARS', 'CREDIT_INCREASED', 'DELTA_CREDIT_PCT_IF_INCREASED', 'DELTA_CREDIT_SUM_IF_INCREASED', 'END_STATUS_SCORE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'HAS_HIGH_RISK_DEBT', 'INST_90PLUS', 'IS_HIGH_INTEREST', 'IS_LONG_TERM_ACTIVE', 'IS_LONG_TERM_LOAN', 'IS_OVERDUE_LONG', 'IS_OVERDUE_SHORT', 'LOAN_HISTORY_YEARS', 'LOAN_HISTORY_YEARS(prev)', 'NO_CREDIT_HISTORY_FLAG', 'NUM_CASH_APPLICATIONS', 'NUM_POS_APPLICATIONS', 'NUM_REVOLVING_APPLICATIONS', 'OVERLAP', 'OVERLAP_YEARS', 'REFUSAL_RATE', 'SK_ID_CURR', 'TARGET', 'avg_payment_delay', 'bureau_DAYS_CREDIT_ENDDATE', 'bureau_IS_ACTIVE_ESTIMATED', 'bureau_IS_CLOSED_ACTUAL', 'pct_full_payments', 'pct_underpaid']


In [76]:
common_cols = set(df_v2.columns) & set(df_v3.columns)
print("공통 컬럼 수:", len(common_cols))

공통 컬럼 수: 37


In [78]:
df_v3

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,CC_90PLUS,AVG_CARD_USAGE_MONTHS,AVG_UTIL_RATIO,AVG_MIN_PAY_RATIO,AVG_REPAY_RATIO,...,DELTA_CREDIT_PCT_IF_INCREASED,OVERLAP,OVERLAP_YEARS,HAS_HIGH_RISK_DEBT,CNT_EARLY_REPAID,CNT_REPAID_ON_TIME,CNT_OVERDUE_ENDED,CNT_NOT_ENDED,CREDIT_AVG_SK_DPD,POS_AVG_SK_DPD
0,100002,0.083037,0.262949,0.139376,1,NaN,NaN,NaN,NaN,NaN,...,4.103036,6.0,9.200000,0.0,3.0,2.0,0.0,3.0,NaN,0.000000
1,100003,0.311267,0.622246,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,10.148963,2.0,-1.032877,0.0,1.0,1.0,1.0,1.0,NaN,0.000000
2,100004,-1.000000,0.555912,0.729567,0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,-0.512329,0.0,1.0,1.0,0.0,0.0,NaN,0.000000
3,100006,-1.000000,0.650442,-1.000000,0,0.0,6.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000
4,100007,-1.000000,0.322738,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0.145570,0.681632,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
307507,456252,-1.000000,0.115992,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
307508,456253,0.744026,0.535722,0.218859,0,NaN,NaN,NaN,NaN,NaN,...,5.250000,3.0,7.013699,0.0,2.0,0.0,0.0,2.0,NaN,0.294118
307509,456254,-1.000000,0.514163,0.661024,1,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,NaN,0.000000


In [82]:
only_in_v2 = set(df_v2.columns) - set(df_v3.columns)
print("v2에만 있는 컬럼 수:", len(only_in_v2))
print(sorted(only_in_v2))

v2에만 있는 컬럼 수: 3
['AVG_SK_DPD_x', 'AVG_SK_DPD_y', 'END_STATUS_SCORE']


In [80]:
# END_STATUS_SCORE 컬럼 삭제
df_v3 = df_v3.drop(columns=["END_STATUS_SCORE"], errors="ignore")

# 저장
df_v3.to_csv("score_data/merged_kpi_dataset_v3.csv", index=False, encoding="utf-8-sig")

print("END_STATUS_SCORE 컬럼 삭제 및 저장 완료 → 'merged_kpi_dataset_v3.csv'")

END_STATUS_SCORE 컬럼 삭제 및 저장 완료 → 'merged_kpi_dataset_v3.csv'


In [48]:
# 원본 bureau
bureau_raw = pd.read_csv("data/bureau.csv")

# 전처리된 bureau_final
bureau_final = pd.read_csv("./score_data/bureau_final.csv")

In [ ]:
# 각각의 고객 ID 집합
id_raw = set(bureau_raw['SK_ID_CURR'])
id_final = set(bureau_final['SK_ID_CURR'])

# 교집합
common_ids = id_raw & id_final

# 차집합 (누락된 것 확인)
only_in_raw = id_raw - id_final
only_in_final = id_final - id_raw

# 출력
print(f"원본에 있는 고객 수: {len(id_raw)}")
print(f"최종본에 있는 고객 수: {len(id_final)}")
print(f"공통 고객 수: {len(common_ids)}")
print(f"원본에만 있고 최종본에 없는 고객 수: {len(only_in_raw)}")
print(f"최종본에만 있고 원본에 없는 고객 수: {len(only_in_final)}")

원본에 있는 고객 수: 305811
최종본에 있는 고객 수: 305811
공통 고객 수: 305811
원본에만 있고 최종본에 없는 고객 수: 0
최종본에만 있고 원본에 없는 고객 수: 0


In [ ]:
# df = df.drop('IS_LOW_POS_ACTIVITY', axis=1)
# df = df.drop(['NUM_TOTAL', 'NUM_REFUSED'], axis = 1)

In [56]:
df = df.drop(['bureau_DAYS_ENDDATE_FACT'], axis = 1)

In [57]:
df

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,CC_90PLUS,AVG_SK_DPD_x,AVG_CARD_USAGE_MONTHS,AVG_UTIL_RATIO,AVG_MIN_PAY_RATIO,...,DELTA_CREDIT_SUM_IF_INCREASED,DELTA_CREDIT_PCT_IF_INCREASED,OVERLAP,OVERLAP_YEARS,HAS_HIGH_RISK_DEBT,HAS_BAD_STATUS_36M,END_STATUS_SCORE,bureau_IS_ACTIVE_ESTIMATED,bureau_IS_CLOSED_ACTUAL,bureau_DAYS_CREDIT_ENDDATE
0,100002,0.083037,0.262949,0.139376,1,NaN,NaN,NaN,NaN,NaN,...,430929.0,4.103036,6.0,9.200000,0.0,1.0,-3.0,1.0,0.750000,-349.0
1,100003,0.311267,0.622246,NaN,0,NaN,NaN,NaN,NaN,NaN,...,737347.5,10.148963,2.0,-1.032877,0.0,NaN,0.0,1.0,0.750000,-544.0
2,100004,NaN,0.555912,0.729567,0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,-0.512329,0.0,NaN,-1.0,0.0,1.000000,-488.0
3,100006,NaN,0.650442,NaN,0,0.0,0.0,6.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,NaN,0.322738,NaN,0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0.000000,0.0,NaN,0.0,0.0,1.000000,-783.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0.145570,0.681632,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,NaN,0.115992,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0.744026,0.535722,0.218859,0,NaN,NaN,NaN,NaN,NaN,...,1890000.0,5.250000,3.0,7.013699,0.0,0.0,-2.0,2.0,0.500000,280.0
307509,456254,NaN,0.514163,0.661024,1,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,-859.0


In [ ]:
# df.to_csv('./score_data/merged_kpi_dataset_v1.csv', index=False, encoding='utf-8-sig')

In [2]:
df = pd.read_csv('./score_data/merged_kpi_dataset_v1.csv')

In [4]:
df['DELTA_CREDIT_PCT_IF_INCREASED'].fillna(-1)

0          4.103036
1         10.148963
2         -1.000000
3         -1.000000
4         -1.000000
            ...    
307506    -1.000000
307507    -1.000000
307508     5.250000
307509    -1.000000
307510    12.407636
Name: DELTA_CREDIT_PCT_IF_INCREASED, Length: 307511, dtype: float64

신용카드(credit_card)
CREDIT_CARD_HISTORY_YEARS
외부대출 (bureau)
LOAN_HISTORY_YEARS
내부대출 (previous)
LOAN_HISTORY_YEARS(prev)

In [6]:
cols = [
    'CREDIT_CARD_HISTORY_YEARS',
    'LOAN_HISTORY_YEARS',
    'LOAN_HISTORY_YEARS(prev)'
]

df[cols] = df[cols].fillna(0)

In [8]:
df[cols]

,CREDIT_CARD_HISTORY_YEARS,LOAN_HISTORY_YEARS,LOAN_HISTORY_YEARS(prev)
0,0.000000,3.9,1.66
1,0.000000,0.0,6.41
2,0.000000,0.0,2.23
3,0.416667,0.0,1.69
4,0.000000,0.0,6.46
...,...,...,...
307506,0.000000,0.0,0.75
307507,0.000000,0.0,6.84
307508,0.000000,2.5,7.81
307509,0.000000,3.0,0.88


In [ ]:
# df.to_csv('./score_data/merged_kpi_dataset_v2.csv', index=False, encoding='utf-8-sig')

In [ ]:
# df = df.drop(['HAS_BAD_STATUS_36M', 'CARD_USAGE_BIN_5P'], axis = 1)

In [18]:
ext_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
df[ext_cols] = df[ext_cols].fillna(-1)

In [21]:
cols = [
    'CREDIT_CARD_HISTORY_YEARS',
    'LOAN_HISTORY_YEARS',
    'LOAN_HISTORY_YEARS(prev)'
]

# 결측치 0으로 채우기 (이미 하셨다면 생략 가능)
df[cols] = df[cols].fillna(0)

# 세 컬럼 모두 0인 경우 1, 아니면 0 → 거래내역 없음 플래그
df['NO_CREDIT_HISTORY_FLAG'] = (df[cols].sum(axis=1) == 0).astype(int)

In [ ]:
df.to_csv('./score_data/merged_kpi_dataset_v2.csv', index=False, encoding='utf-8-sig')

In [25]:
df = pd.read_csv('./score_data/merged_kpi_dataset_v2.csv')

In [26]:
df

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,CC_90PLUS,AVG_SK_DPD_x,AVG_CARD_USAGE_MONTHS,AVG_UTIL_RATIO,AVG_MIN_PAY_RATIO,...,DELTA_CREDIT_SUM_IF_INCREASED,DELTA_CREDIT_PCT_IF_INCREASED,OVERLAP,OVERLAP_YEARS,HAS_HIGH_RISK_DEBT,END_STATUS_SCORE,bureau_IS_ACTIVE_ESTIMATED,bureau_IS_CLOSED_ACTUAL,bureau_DAYS_CREDIT_ENDDATE,NO_CREDIT_HISTORY_FLAG
0,100002,0.083037,0.262949,0.139376,1,NaN,NaN,NaN,NaN,NaN,...,430929.0,4.103036,6.0,9.200000,0.0,-3.0,1.0,0.750000,-349.0,0
1,100003,0.311267,0.622246,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,737347.5,10.148963,2.0,-1.032877,0.0,0.0,1.0,0.750000,-544.0,0
2,100004,-1.000000,0.555912,0.729567,0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,-0.512329,0.0,-1.0,0.0,1.000000,-488.0,0
3,100006,-1.000000,0.650442,-1.000000,0,0.0,0.0,6.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,100007,-1.000000,0.322738,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,1.000000,-783.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0.145570,0.681632,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307507,456252,-1.000000,0.115992,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307508,456253,0.744026,0.535722,0.218859,0,NaN,NaN,NaN,NaN,NaN,...,1890000.0,5.250000,3.0,7.013699,0.0,-2.0,2.0,0.500000,280.0,0
307509,456254,-1.000000,0.514163,0.661024,1,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0.000000,0.0,0.0,0.0,1.000000,-859.0,0


In [27]:
df['DELTA_CREDIT_PCT_IF_INCREASED'] = df['DELTA_CREDIT_PCT_IF_INCREASED'].fillna(-1)

In [28]:
df

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,CC_90PLUS,AVG_SK_DPD_x,AVG_CARD_USAGE_MONTHS,AVG_UTIL_RATIO,AVG_MIN_PAY_RATIO,...,DELTA_CREDIT_SUM_IF_INCREASED,DELTA_CREDIT_PCT_IF_INCREASED,OVERLAP,OVERLAP_YEARS,HAS_HIGH_RISK_DEBT,END_STATUS_SCORE,bureau_IS_ACTIVE_ESTIMATED,bureau_IS_CLOSED_ACTUAL,bureau_DAYS_CREDIT_ENDDATE,NO_CREDIT_HISTORY_FLAG
0,100002,0.083037,0.262949,0.139376,1,NaN,NaN,NaN,NaN,NaN,...,430929.0,4.103036,6.0,9.200000,0.0,-3.0,1.0,0.750000,-349.0,0
1,100003,0.311267,0.622246,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,737347.5,10.148963,2.0,-1.032877,0.0,0.0,1.0,0.750000,-544.0,0
2,100004,-1.000000,0.555912,0.729567,0,NaN,NaN,NaN,NaN,NaN,...,0.0,-1.000000,0.0,-0.512329,0.0,-1.0,0.0,1.000000,-488.0,0
3,100006,-1.000000,0.650442,-1.000000,0,0.0,0.0,6.0,0.0,0.0,...,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,100007,-1.000000,0.322738,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,0.0,-1.000000,0.0,0.000000,0.0,0.0,0.0,1.000000,-783.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0.145570,0.681632,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307507,456252,-1.000000,0.115992,-1.000000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
307508,456253,0.744026,0.535722,0.218859,0,NaN,NaN,NaN,NaN,NaN,...,1890000.0,5.250000,3.0,7.013699,0.0,-2.0,2.0,0.500000,280.0,0
307509,456254,-1.000000,0.514163,0.661024,1,NaN,NaN,NaN,NaN,NaN,...,0.0,-1.000000,0.0,0.000000,0.0,0.0,0.0,1.000000,-859.0,0


In [31]:
print(df['IS_OVERDUE_LONG'].value_counts())

IS_OVERDUE_LONG
0.0    262389
1.0      1056
2.0        33
3.0         9
4.0         2
5.0         2
Name: count, dtype: int64


In [ ]:
df_v3